<a href="https://colab.research.google.com/github/dimash-web/child-health-data-feature/blob/main/Child_health_data_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Feature: Child Health Data

This feature allows a user to pick a metric related to children's welfare, like mortality rate, nutrition, etc. Then, we use the APIs to displays the related welfare stats of all countries on an interactive world map and a tableview.

## We will use two APIs:

The World Health Organization API to access specific metrics of children's welfare around the world.
An API to convert the country codes provided by the WHO into country names.

The API setup does not involve adding API Keys into the SECRETS tab.

## WORLD HEALTH ORGANIZATION API Usage:

Display all indicators related to children's welfare. Allow user to choose 1.
Input the indicator code as a parameter to retrieve the stats
Sort stats from high to low

In [ ]:
#import necessary libraries
import requests
import json
import pandas as pd

#search for all indicators pertaining to "Children" through the filter endpoint to select 1 indicator of interest.
url = f"https://ghoapi.azureedge.net/api/Indicator?$filter=contains(IndicatorName,'Children')"
response = requests.get(url)
if response.status_code == 200:
 data = response.json()
 print("Status: Successful!")
 print(data)
 print(json.dumps(data, indent=4))
else:
  print("Error")

  def display_indicators(data): #allow user to choose their metric of interest through a catalog
  indicators = data['value']
  for idx, indicator in enumerate(indicators):
    print(f"{idx + 1}: {indicator['IndicatorName']}")
  choice = int(input("\nEnter the number of the indicator you want to select: ")) - 1
  if 0 <= choice < len(indicators):
    selected_indicator = indicators[choice]
    print(f"\nYou selected: {selected_indicator['IndicatorName']}")
    print(f"Indicator Code: {selected_indicator['IndicatorCode']}")
    return selected_indicator['IndicatorCode'], selected_indicator['IndicatorName']
  else:
    print("You must choose a valid number!")

indicator_code, indicator_name = display_indicators(data)

indicator_code

# chose "Morality rate among children ages 10 to 14 years (per 1000 children aged 10)"
def get_data(indicator_code):
 url = f'https://ghoapi.azureedge.net/api/{indicator_code}'
 response = requests.get(url)
 if response.status_code == 200:
  print("Status: Successful!")
  data = response.json()
  print(json.dumps(data,indent=4))
  return data
 else:
  print("Error")

data = get_data(indicator_code)

def process_data(data): #process_data is used to
    spatial_values = {}  #create an empty dictionary to later store spatial_dim (key) - numeric_value (value)
    for entry in data['value']: #loop through each country
     #if 'SpatialDim' in entry and 'NumericValue' in entry: #just in case these keys don't appear, but they do appear
     spatial_dim = entry['SpatialDim'] #retrieve the country code (its key name is SpatialDim)
     numeric_value = entry['NumericValue'] #retrieve the child mortality rate
     if spatial_dim not in spatial_values:
       spatial_values[spatial_dim] = numeric_value #append each country code and mortality rate to the dictionary

#sort by mortality rate in descending manner using the dictionary above
    sorted_spatial_values = sorted(spatial_values.items(), key=lambda x: x[1], reverse=True)
    return sorted_spatial_values

#call on the function
sorted_spatial_data = process_data(data)

#testing
for spatial_dim, value in sorted_spatial_data:
  print(f"Country Code: {spatial_dim}, {indicator_name}: {value:.2f}")


## REST COUNTRIES API USAGE:

Convert Country Code into Country Name (since WHO does not provide Country Name)
Then Merge with the output from WHO API.

In [ ]:
country_codes = [] #create an empty list to store country codes
for spatial_dim, _ in sorted_spatial_data: #separate country code from mortality rate in spatial_dim
    country_codes.append(spatial_dim)

print("Country Codes:", country_codes)


def get_countries_name(sorted_spatial_data): #decode country code into country name
    #use the all endpoint to fetch all attributes of a country (including name, code, etc.)
    url = 'https://restcountries.com/v3.1/all'
    response = requests.get(url)
    data = response.json()

    #create a mapping from country codes to country names
    country_code_map = {}
    for country in data:
     country_code_map[country['cca3']] = country['name']['common']
    return country_code_map


def merge_data(sorted_spatial_data): #merge results from 2 apis to get final output
  rows = []
  country_code_map = get_countries_name(sorted_spatial_data)
  for spatial_dim, value in sorted_spatial_data:
    country_name = country_code_map.get(spatial_dim, "Unknown")
    rows.append({'Country Name': country_name, 'Country Code': spatial_dim, indicator_name: value})
    print(f"Country Code: {spatial_dim}, Country Name: {country_name}, {indicator_name}: {value:.2f} ")
  df = pd.DataFrame(rows)
  return df

sorted_spatial_data = process_data(data)
df = merge_data(sorted_spatial_data)

#create a clean tableview of countryname, countrycode, and their respective child moratlity rate
print(df)

#final output
import plotly.express as px
import numpy as np
import pandas as pd

country_codes = df['Country Code']
values = df[indicator_name]

# Create the choropleth map
fig = px.choropleth(
    df,
    locations='Country Code',
    locationmode='ISO-3',  # Use 'ISO-3' for ISO 3166-1 alpha-3 country codes
    color= indicator_name,
    color_continuous_scale='Viridis',
    title= indicator_name
)

# Update layout to enlarge the graph
fig.update_layout(
    width=1400,  # Set width (e.g., 1200 pixels)
    height=800   # Set height (e.g., 800 pixels)
)

# Show the figure
fig.show()

#display df
pd.set_option('display.max_rows', None)  # or a specific number like 1000
pd.set_option('display.max_columns', None)  # or a specific number like 50
df